Load in required libraries, below.

In [1]:
# data 
import pandas as pd 
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, locally stored data can be uploaded to S3.

In [1]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()
prefix='cnn-wendy-data'
prefix_output='cnn-wendy-model'

Here we retrieve the dataset of images and we upload it to S3


In [ ]:
!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data.zip
!unzip -qq -n wendy_cnn_frames_data.zip -d wendy_cnn_frames_data 

In [3]:

# upload to S3. Skip if already uploaded. This can take a while.
print('Uploading data to {}'.format(input_data))
input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data', bucket=bucket, key_prefix=prefix)
print('Data uploaded to {}'.format(input_data))

Model uploaded to s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data


In [4]:
# location to input data can be written down here, if known
# input_data='s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data'

After uploading images to S3, we can define and train the estimator


In [4]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path

output_path = 's3://{}/{}'.format(bucket, prefix_output)
print('Output path for models is {}'.format(output_path))

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier',
                    role=role,
                    framework_version='1.6',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    train_volume_size = 10,
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 96,
                        'img-height': 54,
                        'batch-size': 32,
                        'layer-cfg': 'B',
                        'epochs': 2
                    })

Uploading model to s3://sagemaker-eu-central-1-283211002347/cnn-wendy-model


## Train the Estimator

After instantiating the estimator, we train it with a call to `.fit()`. 

In [ ]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-20 01:50:47 Starting - Starting the training job...
2020-10-20 01:50:50 Starting - Launching requested ML instances......
2020-10-20 01:52:12 Starting - Preparing the instances for training......
2020-10-20 01:53:15 Downloading - Downloading input data....................................
2020-10-20 01:59:12 Training - Downloading the training image...
2020-10-20 01:59:33 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-10-20 01:59:33,401 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-10-20 01:59:33,425 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-10-20 01:59:39,673 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-10-20 01:59:40,034 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/op

In [ ]:
print(estimator.model_data)
model_data = estimator.model_data
# model can be set here, if known
# model_data = 

We set up a model that can predict the class of an image

### Deploy the trained model

We deploy our model to create a predictor. We'll use this to make predictions on our data and evaluate the model.

In [ ]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=model_data,
                     role = role,
                     framework_version='1.6',
                     entry_point='predict.py',
                     source_dir='letsplay_classifier')

In [ ]:
%%time
# deploy and create a predictor
              
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')


In [ ]:
# the endpoint where the predictor is located
endpoint_name = predictor.endpoint


Now that the model is deployed, we check how the predictor performs on our full dataset,
ensuring that the predictions make sense.


In [11]:
print(endpoint_name)

pytorch-inference-2020-10-20-00-22-57-445


In [1]:
endpoint_name='pytorch-inference-2020-10-20-00-22-57-445'
from letsplay_classifier.endpoint import evaluate
avg_acc, avg_loss, count = evaluate(endpoint_name, 'wendy_cnn_frames_data', 0.05)
print("{} processed of {}".format(count, image_total))
print("Avg loss : {:.4f}".format(avg_loss))
print("Avg acc : {:.4f}".format(avg_acc))

tensor([0])
tensor([[ 4.4082,  0.9472,  0.4195,  2.2123, -0.1431, -0.1965,  1.6163,  0.6867,
         -1.6622, -1.6369, -1.5348, -1.4685, -1.3968, -1.5122]])
tensor([[ 7.0574,  2.8788,  2.8210,  2.3834, -2.1640, -0.1867,  0.2931,  0.7889,
         -2.3697, -2.2521, -2.3425, -2.2238, -2.3295, -2.2892]])
tensor([[ 5.9065,  0.7384,  1.4050,  1.5161, -0.5003, -0.2738,  1.9018,  0.4090,
         -1.8510, -1.7856, -1.8723, -1.7893, -1.7173, -1.8275]])
tensor([[ 5.0718, -0.1978,  0.7106,  1.6801, -0.2703,  0.3338,  1.8513,  0.9068,
         -1.6913, -1.6545, -1.5728, -1.4595, -1.4693, -1.5978]])
tensor([[ 4.8330,  0.0914,  1.3228,  1.3610, -0.4521,  0.3514,  0.9924,  0.9539,
         -1.6011, -1.5619, -1.4953, -1.4398, -1.4075, -1.4575]])
tensor([[ 4.7190e+00,  4.5579e-03,  7.5392e-01,  1.5356e+00,  4.0690e-02,
          2.3772e-01,  1.3834e+00,  8.3333e-01, -1.6735e+00, -1.5529e+00,
         -1.4301e+00, -1.4960e+00, -1.3928e+00, -1.5164e+00]])
tensor([[ 4.1397, -0.0160,  0.6040,  1.3927,  0

KeyboardInterrupt: 

## Delete the Endpoint

Finally, I've add a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [24]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [25]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted pytorch-inference-2020-10-19-12-53-25-499
